In [1]:
!pip install -U spacy
!pip install spacy_transformers

  Obtaining dependency information for spacy_transformers from https://files.pythonhosted.org/packages/af/e0/373f02d71ba7ba21629f523d5a05856940deea069c26daf012f809c66e92/spacy_transformers-1.3.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for spacy-alignments<1.0.0,>=0.7.2 from https://files.pythonhosted.org/packages/76/ea/44ffa5dbe1aa412cc274472d5b3dd3171ae2ea6aa2168fc21afa9038b7a9/spacy_alignments-0.9.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.9/197.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 5.9 MB/s eta 0:00:00


In [2]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
spacy.__version__

'3.7.2'

In [4]:
!nvidia-smi

Tue Jan  2 12:42:11 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:00:05.0 Off |                    0 |
| N/A   

In [5]:
jd_data=json.load(open('/kaggle/input/keyword-jd/annotations (5).json'))

In [6]:
jd_data

{'classes': ['MUST HAVE',
  'GOOD TO HAVE',
  'EXPERIENCE',
  'LOCATION',
  'JOB TITLE'],
 'annotations': [['Job Title: AWS Administrator Location: GA, Atlanta, (Remote) Duration: 1 Year Key Responsibilities: Establish configuration, compliance, and audit program to effectively manage AWS accounts and resources. Review cloud resources and cost drivers, compute, storage, network, managed services, database and service licenses, marketplace, and accounts. As part of our Managed Services offering for clients; monitor rightsizing, elasticity, storage optimization, and identify unused resources. Maintain cloud-based servers patching vulnerabilities, backup/restore operations, provision new servers, configure firewalls, configure monitoring systems. Manage account containerization and tagging for internal cost centers and end client billing feeds. Monitor cost allocations, security compliance, budgets and set alerts across accounts, workloads, and users. Operational monitoring to identify an

In [7]:
annotations = jd_data.get('annotations', [])

In [8]:
annotations

[['Job Title: AWS Administrator Location: GA, Atlanta, (Remote) Duration: 1 Year Key Responsibilities: Establish configuration, compliance, and audit program to effectively manage AWS accounts and resources. Review cloud resources and cost drivers, compute, storage, network, managed services, database and service licenses, marketplace, and accounts. As part of our Managed Services offering for clients; monitor rightsizing, elasticity, storage optimization, and identify unused resources. Maintain cloud-based servers patching vulnerabilities, backup/restore operations, provision new servers, configure firewalls, configure monitoring systems. Manage account containerization and tagging for internal cost centers and end client billing feeds. Monitor cost allocations, security compliance, budgets and set alerts across accounts, workloads, and users. Operational monitoring to identify and address issues. Use AWS Config to monitor and track resource configuration. Set standards for resource c

In [9]:
annotations[0]

['Job Title: AWS Administrator Location: GA, Atlanta, (Remote) Duration: 1 Year Key Responsibilities: Establish configuration, compliance, and audit program to effectively manage AWS accounts and resources. Review cloud resources and cost drivers, compute, storage, network, managed services, database and service licenses, marketplace, and accounts. As part of our Managed Services offering for clients; monitor rightsizing, elasticity, storage optimization, and identify unused resources. Maintain cloud-based servers patching vulnerabilities, backup/restore operations, provision new servers, configure firewalls, configure monitoring systems. Manage account containerization and tagging for internal cost centers and end client billing feeds. Monitor cost allocations, security compliance, budgets and set alerts across accounts, workloads, and users. Operational monitoring to identify and address issues. Use AWS Config to monitor and track resource configuration. Set standards for resource co

In [10]:
len(annotations)

50

In [11]:
!python -m spacy init fill-config /kaggle/input/base-config/base_config.cfg /kaggle/working/config.cfg

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
✔ Auto-filled config with all values
✔ Saved config
/kaggle/working/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [12]:
def get_spacy_doc(file,data):
  nlp=spacy.blank('en')
  db= DocBin()
  for text, annot in tqdm(data):
    doc=nlp.make_doc(text)
    annot=annot['entities']

    ents=[]
    entity_indices=[]

    for start, end, label in annot:
      skip_entity= False
      for idx in range(start,end):
        if idx in entity_indices:
          skip_entity=True
          break
      if skip_entity==True:
        continue

      entity_indices=entity_indices+list(range(start,end))

      try:
        span=doc.char_span(start,end,label=label,alignment_mode='strict')
      except:
        continue

      if span is None:
        err_data=str([start,end])+ '  ' + str(text)+'/n'
        file.write(err_data)

      else:
        ents.append(span)
    try:
      doc.ents=ents
      db.add(doc)
    except:
      pass
  return db

In [13]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(annotations,test_size=0.1)

In [14]:
len(train),len(test)

(45, 5)

In [15]:
file=open('/kaggle/working/error.txt','w')
db=get_spacy_doc(file,train)
db.to_disk('/kaggle/working/train_data.spacy')

db=get_spacy_doc(file,test)
db.to_disk('/kaggle/working/test_data.spacy')

file.close()

100%|██████████| 5/5 [00:00<00:00, 196.90it/s]


In [16]:
!python -m spacy train /kaggle/working/config.cfg --output /kaggle/working/ --paths.train /kaggle/working/train_data.spacy --paths.dev /kaggle/working/test_data.spacy --gpu-id 0 

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
ℹ Saving to output directory: /kaggle/working
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2024-01-02 12:42:35,682] [INFO] Set up nlp object from config
[2024-01-02 12:42:35,723] [INFO] Pipeline: ['transformer', 'ner']
[2024-01-02 12:42:35,730] [INFO] Created vocabulary
[2024-01-02 12:42:35,730] [INFO] Finished initializing nlp object
config.json: 100%|█████████████████████████████| 481/481 [00:00<00:00, 2.28MB/s]
vocab.json: 100%|████████████████████████████| 899k/899k [00:00<00:00, 12.3MB/s]
merges.txt: 100%|████████████████████████████| 456k/456k [00:00<00:00, 21.5MB/s]
tokenizer.json: 100%|██████████████████████| 1.36M/1.36M [00:00<00:00, 33.0MB/s]
model.safetensors: 100%|███████████████

In [17]:
nlp=spacy.load('/kaggle/working/model-best')

In [18]:
with open('/kaggle/input/test-data/config.txt') as f:
  text = f.read()

In [19]:
doc = nlp(text)

print(doc.ents)

(Hardware Architect, Machine Learning, Sunnyvale, CA, USA, 8, TensorFlow, PyTorch, TPU, Machine Learning)


In [20]:
for ent in doc.ents:
  print(ent.text, ent.label_)
print(doc.text)

Hardware Architect JOB TITLE
Machine Learning MUST HAVE
Sunnyvale LOCATION
CA LOCATION
USA LOCATION
8 EXPERIENCE
TensorFlow GOOD TO HAVE
PyTorch GOOD TO HAVE
TPU GOOD TO HAVE
Machine Learning MUST HAVE
Hardware Architect, Machine Learning
corporate_fare
Google
place
Sunnyvale, CA, USA
Minimum qualifications:
Bachelor's degree in Computer Science, Electrical Engineering, a related field, or equivalent practical experience.
8 years of experience in computer or chip architecture.
Experience with semiconductor technologies and trends (e.g., process, memory, interconnect, and packaging).

Preferred qualifications:
Experience with learning frameworks including TensorFlow and PyTorch.
Experience building hardware solutions for Machine Learning.
Experience with outreach to Machine Learning researchers and application developers.
Knowledge of Machine Learning market, technological and business trends, software ecosystem, and emerging applications.
About the job
Our computational challenges are 

In [21]:
keywords = []
seen_skills = set()

In [22]:
for ent in doc.ents:
    if ent.label_ in ("MUST HAVE","GOOD TO HAVE", "LOCATION", "EXPERIENCE"):
        if ent.text not in seen_skills:
            keywords.append({"text": ent.text, "label": ent.label_})
            seen_skills.add(ent.text)

In [23]:
with open("keywords.json", "w") as f:
    json.dump(keywords, f, indent=4)

In [24]:
with open("keywords.json", "r") as f:
    data = json.load(f)
for keyword in data:
    print(f"Text: {keyword['text']}, Label: {keyword['label']}")

Text: Machine Learning, Label: MUST HAVE
Text: Sunnyvale, Label: LOCATION
Text: CA, Label: LOCATION
Text: USA, Label: LOCATION
Text: 8, Label: EXPERIENCE
Text: TensorFlow, Label: GOOD TO HAVE
Text: PyTorch, Label: GOOD TO HAVE
Text: TPU, Label: GOOD TO HAVE
